In [ ]:
import pandas as pd
import sqlite3

In [ ]:
def new_connection(dbfile):
    
    """ Function to setup a new connection to a database (SQLite).
    
    Arguments:
    dbfile (char): Specify the name of the database to be created
    
    Returns:
    Connection to database"""
    
    conn=sqlite3.connect(dbfile)
    return conn

In [ ]:
def type_col(col_type):
    
    """ Function to return the type of the column of a dataframe.
    
    Arguments:
    col_type (char): Datatype of the column in a dataframe
    
    Returns:
    Datatype to be created in SQL (this will be concatenated with the string)
    
    """
    if col_type=="int64":
        type1="INTEGER(64)"
    elif col_type=="float64":
        type1="FLOAT(64)"
    else:
        type1="VARCHAR(64)"
    return type1

In [ ]:
def generate_string(tablename, df, primarykey):
    
    """ Function to generate the SQL command as a string. This function can be used when only primary keys have to be created and there are no foreign keys associated.
    
    Arguments:
    tablename (char): Name of the table to be created in the database
    df (dataframe): Parsed dataframe output from which field names for the database will be extracted from the column names
    primarykey (char): Primary unique key associated with the table, this should be one of the columnnames from the dataframe
       
    
    Returns:
    An SQL command as a string which can be executed to create tables
    
    """
    
    columnnames=list(df.columns.values)
    
    sql_string="CREATE TABLE IF NOT EXISTS"+ " "+ tablename + "("
      
    for i in range(len(columnnames)):
        
        if i==len(columnnames)-1:
            col=columnnames[i]
            col_type=df[columnnames[i]].dtype
        
            if col==primarykey:
                coltype=type2(col_type)
                sql_string = sql_string + col + " " + coltype + " " + "PRIMARY KEY" + " " + ")" + ";" 
            else:
                coltype=type2(col_type)
                sql_string= sql_string + col + " " + coltype + " " + ")" + ";"
        
        else:
            
            col=columnnames[i]
            col_type=df[columnnames[i]].dtype
        
            if col==primarykey:
                coltype=type2(col_type)
                sql_string = sql_string + col + " " + coltype + " " + "PRIMARY KEY" + ","
            else:
                coltype=type2(col_type)
                sql_string= sql_string + col + " " + coltype + ","
    
    return (sql_string)

In [ ]:
def generate_string_foreign(tablename, df, primarykey, foreignkey, foreigntable, foreigncolumn):
    
    """ Function to generate the SQL command as a string. This function can be used when a table has to be created with primary and foreign keys.
    
    Arguments:
    tablename (char): Name of the table to be created in the database
    df (dataframe): Parsed dataframe output from which field names for the database will be extracted from the column names
    primarykey (char): Primary unique key associated with the table, this should be one of the columnnames from the dataframe
    foreignkey (char): Name of the column in the table which will be the foreignkey
    foreigntable (char): Name of the parent table which contains the association of the foreignkey 
    foreigncolumn (char): Name of the associated column in the foreigntable 
    
    Returns:
    An SQL command as a string which can be executed to create tables
    
    """
    
    columnnames=list(df.columns.values)
    
    sql_string="CREATE TABLE IF NOT EXISTS"+ " "+ tablename + "("
    
    for i in range(len(columnnames)):
        
        if i==len(columnnames)-1:
            col=columnnames[i]
            col_type=df[columnnames[i]].dtype
        
            if col==primarykey:
                coltype=type2(col_type)
                sql_string = sql_string + col + " " + coltype + " " + "PRIMARY KEY" + " " + ")" + ";"
            elif col==foreignkey:
                coltype=type2(col_type)
                sql_string= sql_string + col + " " + coltype + ","
                sql_string = sql_string + "FOREIGN KEY" + " " + "(" + foreignkey + ")" + " " + "REFERENCES" + " " + foreigntable + " " + "(" + foreigncolumn + ")" + " " + ")" + ";"
                
            else:
                coltype=type2(col_type)
                sql_string= sql_string + col + " " + coltype + " " + ")" + ";"
        
        else:
            
            col=columnnames[i]
            col_type=df[columnnames[i]].dtype
        
            if col==primarykey:
                coltype=type2(col_type)
                sql_string = sql_string + col + " " + coltype + " " + "PRIMARY KEY" + ","
            elif col==foreignkey:
                coltype=type2(col_type)
                sql_string= sql_string + col + " " + coltype + ","
                sql_string = sql_string + "FOREIGN KEY" + " " + "(" + foreignkey + ")" + " " + "REFERENCES" +" " + foreigntable + " " + "(" + foreigncolumn + ")" + ","
            else:
                coltype=type2(col_type)
                sql_string= sql_string + col + " " + coltype + ","
    
    return (sql_string)

In [ ]:
def create_table(database, create_new_table_string):
    
    """ Function to create a new table in SQLite database.
     
    Arguments:
    database (char): Specify the name of the database in which the table has to be created
    create_new_table_string: SQL command to generate the table - spcify the name of table, fields (use generate_string function)
    
    Returns:
    The function creates a table in the specified database
    """
    
    connection = sqlite3.connect(database)
    cursor=connection.cursor()
    cursor.execute(create_new_table_string)
    connection.commit()
    connection.close()

In [ ]:
def push_dataframe(df,database, tablename):
    """ Function to push data in a dataframe into a table in the database.
    
    Arguments:
    df (dataframe): Dataframe which contains data to be pushed into tables
    database (char): Specify name of the database in which the table exists
    tablename (char): Name of the table in the specified database into which data has to be pushed 
    
    Returns:
    The function pushes data in the dataframe into the specified table
    """
    connection = sqlite3.connect(database)
    cursor = connection.cursor()
    df.to_sql(tablename,connection, if_exists="replace")
    connection.commit()
    connection.close()

In [ ]:
def query_data(database, query):
    """ Function to query data from the database.
    
    Arguments:
    database (char): Specify name of the database from which data has to be retrieved
    query (char): SQL command to query data
    
    Returns:
    The function returns the queried data in the dataframe format
    """
    connection = sqlite3.connect(database)
    cursor = connection.cursor()
    
    data=pd.read_sql(query,connection)
    connection.commit()
    connection.close()
    return data